In [ ]:
# %% [markdown]
# # Lab 7: Advanced State Management
#
# **Goal:** Compare different approaches to defining and managing state in LangGraph, moving from the simple `TypedDict` to more robust methods like Pydantic and custom classes.
#
# ---

# %% [markdown]
# ## Part A: TypedDict State (The Basic Approach)
#
# This is the method we've used so far. It's simple and effective for basic use cases.
#
# **Pros:**
# -   Lightweight and built-in to Python.
# -   Easy to understand and read.
#
# **Cons:**
# -   **No runtime validation:** If a node returns the wrong data type, you won't know until another node tries to use it.
# -   **No default values:** Every field must be explicitly set.
# -   **Prone to typos:** A misspelled key (`"decison"` instead of `"decision"`) will cause a runtime error.

# %%
from typing import TypedDict, List

class CandidateStateTypedDict(TypedDict):
    name: str
    resume_text: str
    skills: List[str]
    score: int
    decision: str

# Example usage
state_typed_dict = CandidateStateTypedDict(
    name="John Doe",
    resume_text="Python dev",
    skills=["Python"],
    score=80,
    decision="Pending"
)

print("TypedDict state created:")
print(state_typed_dict)

# %% [markdown]
# ---
# ## Part B: Pydantic State (The Recommended Approach)
#
# Pydantic is a powerful data validation library that integrates seamlessly with LangGraph. It is the recommended approach for any production-level or complex agent.
#
# **Pros:**
# -   **Rich Validation:** Enforces data types, value ranges (`ge=0` means "greater than or equal to 0"), and even regex patterns.
# -   **Default Values:** Fields can have sensible defaults.
# -   **Computed Properties:** You can define properties that are calculated from other state fields (e.g., `is_qualified`).
# -   **Clear Error Messages:** If validation fails, Pydantic gives you a clear error explaining what went wrong.

# %%
# You might need to install pydantic
!pip install pydantic --quiet
print("Pydantic installed.")

from pydantic import BaseModel, Field

class CandidateStatePydantic(BaseModel):
    """A Pydantic model for our state, providing validation and defaults."""
    name: str = Field(default="Unknown", description="The candidate's full name")
    resume_text: str = Field(description="The full, unprocessed text of the resume")
    skills: List[str] = Field(default_factory=list, description="A list of skills extracted from the resume")
    score: int = Field(default=0, ge=0, le=100, description="The candidate's score from 0 to 100")
    decision: str = Field(default="Pending", description="The current hiring decision")

    # A computed property - its value is derived from other fields
    @property
    def is_qualified(self) -> bool:
        """A property that returns True if the score is above the threshold."""
        return self.score >= 60

# %% [markdown]
# ### Testing Pydantic Validation
#
# Let's see what happens when we try to create states with both valid and invalid data.

# %%
# Test 1: Creating a valid state
try:
    valid_state = CandidateStatePydantic(
        name="Jane Smith",
        resume_text="Senior Python and Java developer",
        score=95
    )
    print("Successfully created valid Pydantic state:")
    print(f"Name: {valid_state.name}, Score: {valid_state.score}, Qualified: {valid_state.is_qualified}")
except Exception as e:
    print(f"This should not have failed. Error: {e}")

# Test 2: Creating an invalid state
try:
    print("\nAttempting to create an invalid Pydantic state...")
    invalid_state = CandidateStatePydantic(
        name="Invalid Candidate",
        resume_text="No skills",
        score=150  # This score is out of the valid range (0-100)
    )
except Exception as e:
    print("\nCaught expected validation error:")
    print(e)

# %% [markdown]
# ---
# ## Part C: Custom State Classes (The Most Flexible Approach)
#
# For maximum control, you can define your own Python class to manage state. This allows you to embed logic and methods directly into your state object.
#
# **Pros:**
# -   **Ultimate Flexibility:** You can add any methods you need (e.g., `calculate_score`, `add_skill`).
# -   **Encapsulation:** The logic for manipulating the state is contained within the state object itself.
#
# **Cons:**
# -   **More Boilerplate:** Requires writing more code compared to Pydantic.
# -   **Manual Validation:** You have to write your own validation logic if you need it.

# %%
class CandidateStateCustom:
    """A custom class for managing our recruitment state."""

    def __init__(self, name: str, resume_text: str):
        self.name = name
        self.resume_text = resume_text
        self.skills: List[str] = []
        self.score: int = 0
        self.decision: str = "Pending"
        self.history: List[str] = ["State created"]

    def add_skill(self, skill: str):
        """A method to add a skill and log the action."""
        if skill.title() not in self.skills:
            self.skills.append(skill.title())
            self.history.append(f"Added skill: {skill.title()}")

    def calculate_score(self):
        """A method to calculate the score based on skills."""
        self.score = min(len(self.skills) * 20, 100)
        self.history.append(f"Score calculated: {self.score}")

    def make_final_decision(self):
        """A method to update the final decision based on the score."""
        if self.score >= 60:
            self.decision = "Recommend for Interview"
        else:
            self.decision = "Reject"
        self.history.append(f"Decision made: {self.decision}")

    def get_summary(self) -> dict:
        """Returns a summary dictionary of the current state."""
        return {
            "name": self.name,
            "score": self.score,
            "decision": self.decision,
            "history": self.history
        }

# %% [markdown]
# ### Testing the Custom Class
#
# Let's use the methods we defined on our custom class.

# %%
# Instantiate the custom class
candidate = CandidateStateCustom(name="Alice Johnson", resume_text="An expert in Python and Management.")

# Use the class methods to manipulate the state
candidate.add_skill("Python")
candidate.add_skill("Management")
candidate.add_skill("Leadership")
candidate.calculate_score()
candidate.make_final_decision()

# Print the final summary
print("Final summary from custom state class:")
print(candidate.get_summary())

# %% [markdown]
# ### Comparison Summary
#
# | Method          | Best For                                     | Key Feature                        |
# | --------------- | -------------------------------------------- | ---------------------------------- |
# | **TypedDict** | Simple, quick prototypes                     | Simplicity                         |
# | **Pydantic** | Most production applications, complex states | **Automatic Data Validation** |
# | **Custom Class**| When state needs its own internal logic/methods | Maximum Flexibility & Encapsulation |